In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from plotly.plotly import iplot
import plotly.graph_objs as go
import plotly.io as pio

# plotly.tools.set_credentials_file(username='d116626', api_key='rCLXowBHHQLJHHzTxSNM')
init_notebook_mode(connected=True)

import numpy as np


import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import itertools


import savReaderWriter as sav


In [2]:
import pyreadstat

In [9]:
ano = 2010
file = '{}/{}/{}.sav'.format(RAW_PATH,ano,ano)
file

'/home/d116626/JupyterLab/hexagon/censos/data/raw/2010/2010.sav'

In [33]:
df, meta = pyreadstat.read_sav(file, metadataonly=True)
df_empty, catalog = pyreadstat.read_sav(file, metadataonly=True)


In [34]:
# df, meta2 = pyreadstat.read_sav(file, usecols=["V0001", "V0002"])
df_enriched, meta_enriched = pyreadstat.set_catalog_to_sas(df, meta, catalog, formats_as_category=True)




In [43]:
x = meta_enriched.value_labels


print(x)

{'labels0': {1.0: 'Pessoas', 2.0: 'Emigração', 3.0: 'Mortalidade'}, 'labels1': {'50': ' Mato Grosso do Sul', '11': ' Rondônia', '21': ' Maranhão', '31': ' Minas Gerais', '41': ' Paraná', '51': ' Mato Grosso', '12': ' Acre', '22': ' Piauí', '32': ' Espírito Santo', '42': ' Santa Catarina', '52': ' Goiás', '13': ' Amazonas', '23': ' Ceará', '33': ' Rio de Janeiro', '43': ' Rio Grande do Sul', '53': ' Distrito Federal', '14': ' Roraima', '24': ' Rio Grande do Norte', '15': ' Pará', '25': ' Paraíba', '35': ' São Paulo', '16': ' Amapá', '26': ' Pernambuco', '17': ' Tocantins', '27': ' Alagoas', '28': ' Sergipe', '29': ' Bahia'}, 'labels2': {1.0: ' Região norte (uf=11 a 17)', 2.0: ' Região nordeste (uf=21 a 29)', 3.0: ' Região sudeste (uf=31 a 33 e 35)', 4.0: ' Região sul (uf=41 a 43)', 5.0: ' Região centro-oeste (uf=50 a 53)'}, 'labels3': {1.0: ' Urbana', 2.0: ' Rural'}, 'labels4': {1.0: ' Pessoa responsável pelo domicílio', 2.0: ' Cônjuge ou companheiro(a) de sexo diferente', 3.0: ' Cônjug

In [81]:
df_enriched = pyreadstat.set_value_labels(df, meta, formats_as_category=True)

In [82]:
df_enriched.columns = meta.column_labels

In [83]:
df_enriched.head(3)

,UF,MUNICÍPIO,None,Situação,SEXO,RELAÇÃO COM CHEFE,TIPO DE IDADE - MESES OU ANOS,IDADE - MESES/ANOS,RELIGIÃO,COR,NATURALIDADE,NACIONALIDADE,PROCEDÊNCIA,TEMPO DE IMIGRAÇÃO,LUGAR RESIDÊNCIA ANTERIOR,ALFABETIZAÇÃO,ÚLTIMA SÉRIE CONCLUIU,GRAU DO CURSO,CURSO COMPLETO,ESTADO CONJUGAL,ANO DO CASAMENTO,FILHOS TIDOS,FILHOS VIVOS,RENDIMENTOS,ATIVIDADE NÃO ECONÔMICA,OCUPAÇÃO HABITUAL,OCUPAÇÃO NA ULTIMA SE,RAMO E CLASSE DE ATI,POSIÇÃO NA OCUPAÇÃO,None,Situação,ESPÉCIE DO DOMICÍLIO,TIPO DO DOMICÍLIO,CONDIÇÃO DE OCUPAÇÃO,ALUGUEL MENSAL,ABASTECIMENTO DE ÁGUA,INSTALAÇÃO SANITÁRIA,FOGÃO,ILUMINAÇÃO ELÉTRICA,RÁDIO,GELADEIRA,TELEVISÃO,TOTAL DE COMODOS,PEÇAS SERVINDO DOMITÓRIO
0,Rondonia,11.0,2.0,URBANO,MULHER PRESENTE,CHEFE,Anos,60.0,CATÓLICA ROMANA,PARDA,MARANHÃO,BRASILEIRO NATO,URBANA,11 ANOS E MAIS,ACRE,LÊ E NÃO VAI ESCOLA,SEGUNDA SÉRIE,ELEMENTAR,SEM CURSO COMPLETO,VIUVO,NÃO TEM,SEM FILHOS,0.0,DE 3 301 A 4 500,APOSENTADO,NaN,NaN,NaN,NaN,1.0,URBANO,DOMIC PART ÚNICO,DURÁVEL,PRÓPRIO,NÃO PAGA ALUGUEL,REDE C/ CAN INT,FOSSA ASSÉPTICA,CARVÃO,TEM,NÃO TEM,NÃO TEM,NÃO TEM,6.0,2.0
1,Rondonia,11.0,3.0,URBANO,MULHER PRESENTE,AGREGADO,Anos,16.0,CATÓLICA ROMANA,PARDA,MATO GROSSO,BRASILEIRO NATO,URBANA,11 ANOS E MAIS,ACRE,LÊ E FREQÜENTA ESCOLA,QUINTA SÉRIE,ELEMENTAR,PRIMÁRIO,SOLTEIRO,NÃO TEM,SEM FILHOS,0.0,NÃO TEM,ESTUDANTE,NaN,NaN,NaN,NaN,1.0,URBANO,DOMIC PART ÚNICO,DURÁVEL,PRÓPRIO,NÃO PAGA ALUGUEL,REDE C/ CAN INT,FOSSA ASSÉPTICA,CARVÃO,TEM,NÃO TEM,NÃO TEM,NÃO TEM,6.0,2.0
2,Rondonia,11.0,2.0,URBANO,MULHER PRESENTE,CHEFE,Anos,24.0,CATÓLICA ROMANA,PARDA,AMAZONAS,BRASILEIRO NATO,URBANA,11 ANOS E MAIS,NATURAL/NÃO MORA,LÊ E NÃO VAI ESCOLA,QUARTA SÉRIE,MÉDIO PRIM. CICLO,GINASIAL,SOLTEIRO,NÃO TEM,SEM FILHOS,0.0,DE 4 501 A 6 000,PREJUDICADO (ECO,DATILÓGRAFOS,OCUPAÇÃO DECLARADA,SERVIÇO ADIMINSIT,EMPREGADO PUBLIC,2.0,URBANO,DOMIC PART ÚNICO,DURÁVEL,ALUGADO,DE 1001 A 2000,REDE C/ CAN INT,REDE DE ESGOTO,CARVÃO,TEM,NÃO TEM,NÃO TEM,NÃO TEM,6.0,1.0
